In [1]:
import os
from datetime import datetime
from typing import Literal

import torch
import wandb
import numpy as np
import tensorflow as tf
import tensorflow_ranking as tfr
from torch import nn
from torch import optim
import torch_geometric.nn as geonn
from torch_geometric.data import Data, Batch
from tqdm import tqdm

from gfos.data.utils import load_layout
from gfos.data.dataset import LayoutDataset
from gfos.metrics import kendall, topk_error
from gfos.loss import MultiElementRankLoss, listMLE
from gfos.utils.misc import seed_everything


SEED = 42
DEBUG = False

seed_everything(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"

os.environ["WANDB_NOTEBOOK_NAME"] = "layout_gnn_gnn.ipynb"


## Configs

In [2]:
LAYOUT_DIR = r"H:\data\gfos\predict-ai-model-runtime\npz_all\npz\layout"

configs = dict(
    conv_layer="SAGEConv",
    num_epochs=3000,
    learning_rate=5e-3,
    weight_decay=1e-7,
    min_lr=1e-7,
    warmup_ratio=0.0,
    max_configs=2000,
    num_configs=8,
    graph_hidden=[16, 32, 16, 48],
    num_encoder=1,
    num_feedforward=512,
    nhead=1,
    loss_margin=0.5,
    loss_num_permutations=50,
)

WANDB_PROJECT = "gfos"
WANDB_DIR = "../../logs/"
WANDB_RUN_NAME = "layout_xla_default"
TAGS = ["train", "layout", "xla", "default"]

NUM_VAL_EPOCHS = 100
INFERENCE_CONFIGS_BATCH_SIZE = 10
TOP_K = 500

## Model

In [3]:
def aggregate_neighbors(node_feat: torch.Tensor, edge_index: torch.Tensor):
    source_nodes = edge_index[0]
    target_nodes = edge_index[1]

    in_degree_features = torch.zeros_like(node_feat, device=node_feat.device)
    out_degree_features = torch.zeros_like(node_feat, device=node_feat.device)

    source_node_features = node_feat[source_nodes]
    target_node_features = node_feat[target_nodes]

    in_degree_features.scatter_reduce_(
        0,
        target_nodes.unsqueeze(-1).expand_as(source_node_features),
        source_node_features,
        reduce="mean",
        include_self=False,
    )

    out_degree_features.scatter_reduce_(
        0,
        source_nodes.unsqueeze(-1).expand_as(target_node_features),
        target_node_features,
        reduce="mean",
        include_self=False,
    )

    return out_degree_features - in_degree_features

In [4]:
class LayoutModel(torch.nn.Module):
    def __init__(
        self,
        conv_layer: Literal["GATConv", "GCNConv", "SAGEConv"],
        hidden_channels: list[int],
    ):
        super().__init__()

        conv_layer = getattr(geonn, conv_layer)

        op_embedding_dim = 32
        config_dim = 64
        graph_out = 64
        merged_node_dim = graph_out + config_dim

        self.embedding = torch.nn.Embedding(
            120,
            op_embedding_dim,
        )
        assert len(hidden_channels) > 0
        in_channels = op_embedding_dim + 140
        self.convs = geonn.Sequential('x, edge_index', [
            (conv_layer(in_channels, 32), 'x, edge_index -> x1'),
            nn.LeakyReLU(inplace=True),
            (conv_layer(32, 32), 'x1, edge_index -> x2'),
            nn.LeakyReLU(inplace=True),
            (lambda x1, x2: x1 + x2, 'x1, x2 -> x3'),
            (conv_layer(32, 64), 'x3, edge_index -> x4'),
            nn.LeakyReLU(inplace=True),
            (conv_layer(64, 64, normalize=True, bias=False), 'x4, edge_index -> x5'),
        ])
            

        self.config_convs = geonn.Sequential('x, edge_index', [
            (nn.Dropout(p=0.2), 'x -> x'),
            (conv_layer(merged_node_dim, 32), 'x, edge_index -> x1'),
            nn.LeakyReLU(inplace=True),
            (conv_layer(32, 32), 'x1, edge_index -> x2'),
            nn.LeakyReLU(inplace=True),
            (lambda x1, x2: x1 + x2, 'x1, x2 -> x3'),
            (nn.Dropout(p=0.2), 'x3 -> x3'),
            (conv_layer(32, 64), 'x3, edge_index -> x4'),
            nn.LeakyReLU(inplace=True),
            (conv_layer(64, 64, normalize=True), 'x4, edge_index -> x5'),
            # nn.LeakyReLU(inplace=True),
            # (lambda x4, x5: x4 + x5, 'x4, x5 -> x6'),
            # (conv_layer(64, 64), 'x6, edge_index -> x7'),
            # nn.LeakyReLU(inplace=True),
        ])

        self.config_prj = nn.Sequential(
            nn.Linear(18, config_dim),
            # nn.LayerNorm(config_dim),
            nn.LeakyReLU(),
        )
        
        # self.deg_prj = nn.Sequential(
        #     nn.Linear(hidden_channels[-1], merged_node_dim, bias=False),
        #     nn.LayerNorm(merged_node_dim),
        #     nn.LeakyReLU(),
        # )

        # Define a sequential dense neural network
        self.dense = torch.nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(64, 64, bias=False),
            nn.LeakyReLU(),
            nn.Linear(64, 64, bias=False),
            nn.LeakyReLU(),
            nn.Linear(64, 1, bias=False),
        )

    def forward(
        self,
        node_feat: torch.Tensor,
        node_opcode: torch.Tensor,
        edge_index: torch.Tensor,
        node_config_feat: torch.Tensor,
        node_config_ids: torch.Tensor,
        config_edge_index: torch.Tensor,
    ) -> torch.Tensor:
        # Get graph features
        c = node_config_feat.size(0)

        x = torch.cat([node_feat, self.embedding(node_opcode)], dim=1)

        # Get graph features
        x = self.convs(x, edge_index)

        # neighbor_feat = aggregate_neighbors(x, edge_index)[node_config_ids]
        
        # (N, graph_out) -> (NC, graph_out)
        x = x[node_config_ids]
        # x += neighbor_feat

        # Merge graph features with config features
        # (C, NC, 18) -> (C, NC, config_dim)
        node_config_feat = self.config_prj(node_config_feat)
        # pos_embedding = self.deg_prj(neighbor_feat)

        # (C, NC, graph_out + config_dim)
        x = torch.cat([x.repeat((c, 1, 1)), node_config_feat], dim=-1)
        # x += pos_embedding
        x = nn.functional.normalize(x, dim=-1)
        # x = x.permute([1, 0, 2])
        # print(x.shape)
        
        datas = [Data(x=x[i], edge_index=config_edge_index) for i in range(x.shape[0])]
        batch = Batch.from_data_list(datas)

        x = self.config_convs(batch.x, batch.edge_index)
        x = geonn.pool.global_mean_pool(x, batch.batch)

        x = self.dense(x).flatten()

        return x

## Prepare for training

### Expand configs

In [5]:
conv_layer = configs["conv_layer"]
num_epochs = configs["num_epochs"]
learning_rate = configs["learning_rate"]
weight_decay = configs["weight_decay"]
min_lr = configs["min_lr"]
warmup_ratio = configs["warmup_ratio"]
max_configs = configs["max_configs"]
num_configs = configs["num_configs"]
graph_hidden = configs["graph_hidden"]
num_encoder = configs["num_encoder"]
num_feedforward = configs["num_feedforward"]
nhead = configs["nhead"]
margin = configs["loss_margin"]
number_permutations = configs["loss_num_permutations"]


## Training

In [21]:
layout_random = load_layout(
    LAYOUT_DIR,
    model_type="xla",
    compile_type="random",
)
test_dataset = LayoutDataset(files=layout_random["valid"], config_edges=True)

conv_layer = configs["conv_layer"]
num_epochs = configs["num_epochs"]
learning_rate = configs["learning_rate"]
weight_decay = configs["weight_decay"]
min_lr = configs["min_lr"]
warmup_ratio = configs["warmup_ratio"]
max_configs = configs["max_configs"]
num_configs = configs["num_configs"]
graph_hidden = configs["graph_hidden"]
num_encoder = configs["num_encoder"]
num_feedforward = configs["num_feedforward"]
nhead = configs["nhead"]
margin = configs["loss_margin"]
number_permutations = configs["loss_num_permutations"]

model = LayoutModel(
    conv_layer=conv_layer,
    hidden_channels=graph_hidden,
).to(device)


static_dict = torch.load("../../logs/wandb/run-20231002_023625-3tbehqa3/files/2599_0.4760.pth", map_location=device)
model.load_state_dict(static_dict)
model = model.eval()


Loading data: 100%|██████████| 7/7 [00:28<00:00,  4.04s/it]


In [22]:
layout_xla_scores = []
pred_idx = []
results = {}
_INFERENCE_CONFIGS_BATCH_SIZE = 10
scores = []

with torch.no_grad():
    for data in tqdm(test_dataset):
        model_id = data["model_id"]
        node_feat = data["node_feat"]
        node_opcode = data["node_opcode"]
        edge_index = data["edge_index"]
        node_config_feat = data["node_config_feat"]
        node_config_ids = data["node_config_ids"]
        config_edges = data["config_edge_index"]
        config_runtime = data["config_runtime"]

        (
            node_feat,
            node_opcode,
            edge_index,
            node_config_feat,
            node_config_ids,
            config_edges,
        ) = (
            node_feat.to(device),
            node_opcode.to(device),
            edge_index.to(device),
            node_config_feat.to(device),
            node_config_ids.to(device),
            config_edges.to(device),
        )

        outs = []

        c = node_config_feat.size(0)

        for i in range(0, c, _INFERENCE_CONFIGS_BATCH_SIZE):
            end_i = min(i + _INFERENCE_CONFIGS_BATCH_SIZE, c)
            out: torch.Tensor = model(
                node_feat,
                node_opcode,
                edge_index,
                node_config_feat[i:end_i],
                node_config_ids,
                config_edges,
            )
            outs.append(out.detach().cpu())
        outs = torch.cat(outs)
        scores.append(kendall(np.argsort(outs), np.argsort(config_runtime)))

        pred_idx = np.argsort(outs.numpy())

        results[model_id] = pred_idx.tolist()

100%|██████████| 7/7 [01:24<00:00, 12.12s/it]


In [23]:
kendall(outs, config_runtime),\
kendall(np.argsort(outs), np.argsort(config_runtime))

(0.2463617574461254, 0.23418225918225924)

In [ ]:
with open("../../output/xla_random_3tbehqa3_v2.csv", "w") as f:
    f.write("ID,TopConfigs\n")
    for k, v in results.items():
        model_id = "layout:xla:random:"+k
        values = ";".join([str(i) for i in v])
        f.write(f"{model_id},{values}\n")
